# Description

This notebook only illustrates the workings of the auxiliary TS_Mixer model including static and dynamic features.

# Imports

In [1]:
from TSMixer.model_auxiliary import *

# Sample Data

In [2]:
torch.manual_seed(123)
fcst_h = 20
out_dim = 2
x = torch.randn(100,3)[None,:]
z = torch.randn(fcst_h,2)[None,:]
s = torch.ones_like(z)
s[:,:, 1] = 2

In [3]:
n_ts = x.shape[2]
n_static_feat = s.shape[2]
n_dynamic_feat = z.shape[2]
ts_length = x.shape[1]
embed_dim = 64

n_feat_sx = embed_dim + n_ts
n_feat_sz = embed_dim + n_dynamic_feat

# TS-Mixer

In [11]:
num_blocks=2
out_dim=2
ts_mixer_aux = TS_Mixer_auxiliary(n_ts, n_static_feat, n_dynamic_feat, ts_length, embed_dim, num_blocks, fcst_h, out_dim, dropout=0.1)
out = ts_mixer_aux(x, z, s)
print(ts_mixer_aux)

TS_Mixer_auxiliary(
  (fc_map): Linear(in_features=100, out_features=20, bias=True)
  (mlp_sx): MLP_Feat(
    (mlp1): Sequential(
      (0): Linear(in_features=2, out_features=64, bias=True)
      (1): ReLU()
      (2): Dropout(p=0.1, inplace=False)
    )
    (mlp2): Sequential(
      (0): Linear(in_features=64, out_features=64, bias=True)
      (1): Dropout(p=0.1, inplace=False)
    )
    (projector): Linear(in_features=2, out_features=64, bias=True)
    (bn): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (mlp_sz): MLP_Feat(
    (mlp1): Sequential(
      (0): Linear(in_features=2, out_features=64, bias=True)
      (1): ReLU()
      (2): Dropout(p=0.1, inplace=False)
    )
    (mlp2): Sequential(
      (0): Linear(in_features=64, out_features=64, bias=True)
      (1): Dropout(p=0.1, inplace=False)
    )
    (projector): Linear(in_features=2, out_features=64, bias=True)
    (bn): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_